### Function to search for a keyword in a page

In [1]:
#Finds matches for search term in a page
from bs4 import BeautifulSoup
import requests
import re
import sys
import urllib
from urllib.request import Request, urlopen

def search_url(url, search_term):
  
  # print(url.split("www.")[-1].split("//")[-1].split(".")[0])
  # html = urllib.urlopen(url).read()
  #print (url)
  link=url
  if sys.version_info[0] == 3:
      from urllib.request import urlopen
  else:
      # Not Python 3 - today, it is most likely to be Python 2
      # But note that this might need an update when Python 4
      # might be around one day
      from urllib import urlopen

  try:  
    url=Request(url, headers={'User-Agent': 'Mozilla/5.0}'})
    # Your code where you can use urlopen
    with urlopen(url) as url:
        s = url.read()
  except:
    return 0
  # print(s)

  soup = BeautifulSoup(s, 'html.parser')

  # kill all script and style elements
  for script in soup(["script", "style"]):
      script.extract()    # rip it out

  # get text
  text = soup.get_text()

  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)

  # print(text)
  search_term=search_term.lower()
  #text=text.lower()
  matches=0
  full_extract=""
  matched_sentences=[]

  for line in text.split("\n"):
    if len(line.split(" "))>20:
      for sentence in line.split('.'):
        for match in [m.start() for m in re.finditer(search_term, sentence.lower())]:
            #print (sentence+".")
            matched_sentences.append(sentence+".")
            #print()
            matches=matches+1        
        
  print("Matches in this url - "+str(matches))   
  return({
      "url":link,
      "matches":matches,
      "matched_sentences":matched_sentences
  })

# url="https://www.indiatoday.in/india/story/india-china-military-level-talks-ladakh-depsang-situation-1709067-2020-08-08"
# search_term="Ladakh"
# search_url(url, search_term)

### Function to search for links present in a page

This is an unfiltered sample of links found
/health-injuries/a20846808/can-i-run-on-a-swollen-knee-if-it-doesnt-hurt/
/runners-knee/
/health-injuries/

`//www.runnersworld.com/health-injuries/a19578964/patellofemoral-pain-syndrome/`
<br></br></br>
`/runners-knee/`
<br>
`/health-injuries/`
<br>
`//www.runnersworld.com/video/a20788823/prevent-knee-pain/`
<br>
`https://www.runnersworld.com/health-injuries/a22813018/soleus-muscle-pain/`
<br>
`//www.runnersworld.com/health-injuries/a20792564/prevent-and-treat-bunions-com/`
<br>


Some are pretty obvious, but some are less so. Like:
* /health-injuries/a20846808/can-i-run-on-a-swollen-knee-if-it-doesnt-hurt/ ---> https://www.runnersworld.com/health-injuries/a20846808/can-i-run-on-a-swollen-knee-if-it-doesnt-hurt/
<br><br>
* //www.runnersworld.com/health-injuries/a20797170/treating-and-preventing-iliotibial-band-syndrome/ ---> www.runnersworld.com/health-injuries/a20797170/treating-and-preventing-iliotibial-band-syndrome/


In [5]:
#Reruns list of URLs found in a page
def get_urls(url_og):
    all_url_count=0
    url_count=0
    url_list=[]

    try:
        r  = requests.get(url_og)
        data = r.text
        soup = BeautifulSoup(data)    

        for link in soup.find_all('a'):
            all_url_count=all_url_count+1
        #[(all_url_count=all_url_count+1) for link in soup.find_all('a')]
        print("All URL count is "+str(all_url_count))

        # regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
        for link in soup.find_all('a'):
            url=link.get('href')
            if (url):
                if bool(re.search(r'^(http|https|www)', url)):
                    #r'^(http|https|www)'
                    #print(url)
                    url_list.append(url)
                    url_count=url_count+1
                elif bool(re.search(r'^//', url)):
                    #print(url[2:])
                    url_list.append(url[2:])
                    url_count=url_count+1                    
                elif bool(re.search(r'^/[a-z]', url)):
                    #print(url_og.split('/')[2]+url)
                    url_list.append(url_og.split('/')[2]+url)
                    url_count=url_count+1                    

    except:
        pass
        
            
    print("Filtered URL count is "+str(url_count))
    
    url_list = list(dict.fromkeys(url_list))    
    #print(url_list)
    print("Duplicates removed URL count is "+str(len(url_list)))
    return(url_list)

# url="https://www.runnersworld.com/health-injuries/a20823869/knee-injuries-from-running/"
# url="https://www.indiatoday.in/india/story/india-china-military-level-talks-ladakh-depsang-situation-1709067-2020-08-08"
# get_urls(url)

In [3]:
# url = raw_input("Enter a website to extract the URL's from: ")
# url="https://medium.com/@dmahugh_70618/deploying-a-flask-app-to-google-app-engine-faa883b5ffab"
# url="https://matadornetwork.com/notebook/how-to-travel-by-cargo-ship/"
# url="https://www.thehindubusinessline.com/opinion/columns/r-srinivasan/india-incs-living-in-denial/article32278690.ece?__sta=vhg.uosvpxezzlbsksaq%7CFYU&__stm_medium=email&__stm_source=smartech&utm_source=ThePrint&utm_medium=Email&utm_campaign=TFPMailer&utm_content=8Aug2020"

source_url="https://www.indiatoday.in/india/story/india-china-military-level-talks-ladakh-depsang-situation-1709067-2020-08-08" 
search_term="ladakh"

# source_url="https://en.wikipedia.org/wiki/Tom_Crean_(explorer)"
# search_term="antarctica"

# source_url="https://en.wikipedia.org/wiki/Central_Intelligence_Agency"
# search_term="cuba"

# source_url="https://science.sciencemag.org/content/359/6380/1146"
# search_term="virality" Not an idea search term

### Logic for this notebook

It's pretty simple really:
1. Take the `home page` and `depth` from the user. 
2. Search through all the links in the `page`
3. Further process (i.e loop back to the above step) only those links that have one or more match
4. Keep doing this till you have reached the specified depth

In [14]:
from bs4 import BeautifulSoup
import requests
import re
import time
import datetime
start_time=datetime.datetime.now()
print(start_time)

depth=1
print("Depth set as "+str(depth))
articles_searched=0
links_parsed=[]
captured_matches={}
def search_driver(source_url, level):
    level=level+1
    
    try:
        captured_matches[str(level)]
        #print(captured_matches[str(level)])
    except:
        
        captured_matches[str(level)]=[]
    
    if level<=depth:
        print("    "*level+"LEVEL is "+str(level))    
        print("SOURCE URL IS "+source_url)
        
        
        match_dict=search_url(source_url, search_term)
        if match_dict!=0 and match_dict["matches"]!=0: #ONly dig into links that have one or more matches
            print(match_dict)    
            captured_matches[str(level)].append(match_dict)
            if (level+1<=depth): #So that we don't waste time extracting URLs that are beyond required depth
                for url in get_urls(source_url):
                    global links_parsed
                    if url not in links_parsed: #Remove lnks that have already been parsed earlier
                        #print("in loop level is "+str(level)+" "+url)
                        links_parsed.append(url)
                        global articles_searched
                        articles_searched=articles_searched+1
                        print()
                        search_driver(url,level)

                    
search_driver(source_url, -1)
print("\nArticles_searched "+str(articles_searched))
print("Articles_searched "+str(len(links_parsed)))

for depth in captured_matches.keys():
    print("DEPTH - "+depth)
    for url_dict in captured_matches[depth]:
        print("NUMBER of matches for "+ url_dict['url'] +" are "+str(url_dict['matches']))
        for sentence in url_dict['matched_sentences']:
            print(sentence)
            print()
    print("------------------------------------------------------------")

end_time=datetime.datetime.now()
print(end_time)
print("TIME taken is "+str(end_time-start_time))

2020-08-13 02:59:52.696404
Depth set as 1
LEVEL is 0
SOURCE URL IS https://www.indiatoday.in/india/story/india-china-military-level-talks-ladakh-depsang-situation-1709067-2020-08-08
Matches in this url - 5
{'url': 'https://www.indiatoday.in/india/story/india-china-military-level-talks-ladakh-depsang-situation-1709067-2020-08-08', 'matches': 5, 'matched_sentences': ['India, China to hold military-level talks today in Ladakh, to discuss Depsang situationIndia and China will hold Major General-level talks on Saturday at Daulat Beg Oldi area to discuss disengagement by the Chinese side along the Line of Actual Control in Ladakh sector, Indian Army sources said.', 'India, China to hold military-level talks today in Ladakh, to discuss Depsang situationIndia and China will hold Major General-level talks on Saturday at Daulat Beg Oldi area to discuss disengagement by the Chinese side along the Line of Actual Control in Ladakh sector, Indian Army sources said.', '(Photo: PTI/Representational im

Matches in this url - 0

    LEVEL is 1
SOURCE URL IS https://www.youtube.com/channel/UCYPvAwZP8pZhSMW8qs7cVCw?sub_confirmation=1
Matches in this url - 0

    LEVEL is 1
SOURCE URL IS https://web.whatsapp.com:/send?text=India%2C%20China%20to%20hold%20military-level%20talks%20today%20in%20Ladakh%2C%20to%20discuss%20Depsang%20situation%20https%3A%2F%2Fwww.indiatoday.in%2Findia%2Fstory%2Findia-china-military-level-talks-ladakh-depsang-situation-1709067-2020-08-08%20%0A%0ADownload%20the%20India%20Today%20app%20now%20to%20read%20our%20latest%20stories.%0Ahttps%3A%2F%2Findiatoday.app.link%2FTrK1ggqhLT

    LEVEL is 1
SOURCE URL IS https://www.indiatoday.in/india/story/india-discuss-disengagement-china-only-pangong-finger-area-ladakh-1707070-2020-08-02
Matches in this url - 8
{'url': 'https://www.indiatoday.in/india/story/india-discuss-disengagement-china-only-pangong-finger-area-ladakh-1707070-2020-08-02', 'matches': 8, 'matched_sentences': ["India to discuss disengagement by China only in P

Matches in this url - 1
{'url': 'https://www.indiatoday.in/programme/world-today/video/india-pakistan-china-nepal-how-cartographic-changes-have-become-new-diplomatc-tool-1709241-2020-08-08?utm_source=rhs&utm_medium=It&utm_campaign=watchrightnow&t_source=rhs&t_medium=It&t_campaign=watchrightnow', 'matches': 1, 'matched_sentences': [' While the new map reiterated their earlier claims over regions of Jammu and Kashmir and Ladakh as Pakistani territories, a new and somewhat surprising claim was made over Junagadh and, to some extent, over Sir Creek in Gujarat.']}

    LEVEL is 1
SOURCE URL IS https://www.indiatoday.in/india/video/mood-of-the-nation-prime-minister-narendra-modi-nda-king-indian-politics-finds-poll-1709255-2020-08-08?utm_source=rhs&utm_medium=It&utm_campaign=watchrightnow&t_source=rhs&t_medium=It&t_campaign=watchrightnow
Matches in this url - 0

    LEVEL is 1
SOURCE URL IS https://www.indiatoday.in/india/video/kerala-rains-imd-issues-red-alert-for-malappuram-orange-alert-in-

## OBSERVATIONS
### For IndiaToday
Around 15-20% URLs repeated on the page itself
source_url="https://www.indiatoday.in/india/story/india-china-military-level-talks-ladakh-depsang-situation-1709067-2020-08-08" 
search_term="ladakh"
* Depth=1, Time:40sec, Articles_searched:68;
* Depth=2, Time:47min, Articles_searched:3933(=68+3865);
* Depth=2, Time:1m57sec, Articles_searched:174, With optimizations;
* Depth=2, Time:1m15sec, Articles_searched:207, With extra links?;

### For Wikipedia
source_url="https://en.wikipedia.org/wiki/Tom_Crean_(explorer)"
search_term="antarctica"
* Depth=1, Time:2min, Articles_searched:52;
* Depth=2, Time:, Articles_searched:;

source_url="https://en.wikipedia.org/wiki/Central_Intelligence_Agency"
search_term="cuba"
Around 90% URLs repeated on the page itself
* Depth=1, Time16.5min:, Articles_searched:353;
* Depth=2, Time:2h20min, Articles_searched:2862, Interrepted;
* Depth=2, Time:59min, Articles_searched:3815, Optimised;

All URL count is 3111;
Filtered URL count is 2411;
Duplicates removed URL count is 2011.

### For Research paper
Around 85% URLs repeated on the page itself

source_url="https://en.wikipedia.org/wiki/Tom_Crean_(explorer)"
search_term="antarctica"
* Depth=1, Time:2.5min, Articles_searched:55;
* Depth=2, Time:1h20min, Articles_searched:684;
* Depth=2, Time:3min34sec, Articles_searched:50, with optimizations;

